In [1]:
import pandas as pd
import time
import re
import numpy as np
import math

original query sra:
(("Arabidopsis thaliana"[Organism] OR ("Arabidopsis thaliana"[Organism] OR ("Arabidopsis thaliana"[Organism] OR Arabidopsis thaliana[All Fields]))) AND ("strategy rna seq"[Properties] OR "strategy other"[Properties])) AND "Arabidopsis thaliana"[Organism]

In [2]:
#Input path
path_tablajose = '/home/joaquin/projects/transcriptomeDatabase/getting_tables_sra_v2/tables/DatabaseSamplesJOSE.csv'
path_tablasra = '/home/joaquin/projects/transcriptomeDatabase/getting_tables_sra_v2/tables/SraRunInfo.csv'
path_tablaweb = '/home/joaquin/projects/transcriptomeDatabase/getting_tables_sra_v2/tables/SRAinformation.tsv'
path_tablaena = '/home/joaquin/projects/transcriptomeDatabase/getting_tables_sra_v2/tables/definitivo.txt'
path_infotable = '/home/joaquin/projects/transcriptomeDatabase/getting_tables_sra_v2/tables/sra_result.csv'
#Upload files
dfjose = pd.read_csv(path_tablajose)
dfsra = pd.read_csv(path_tablasra)
dfena = pd.read_csv(path_tablaena, sep ='\t')
dfweb = pd.read_csv(path_tablaweb,sep='\t')
dfinfo = pd.read_csv(path_infotable)

/home/joaquin/software/miniconda/envs/mprnaseq/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3257: DtypeWarning: Columns (40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:


dfinfo.rename(columns={"Experiment Accession": "Experiment"},inplace = True)
# TODO REVISAR ESTA QUERY A SRA
dfsra_merge_dfinfo =  pd.merge(dfsra, dfinfo, how='outer', on='Experiment')

# TODO REVISAR ESTA QUERY A SRA

dfena.rename(columns = {'run_accession':'Run'}, inplace = True)
dfena_nodup = dfena.drop_duplicates()
dfsra_dfinfo_merge_dfena_nodupl =  pd.merge(dfsra_merge_dfinfo, dfena_nodup, how='outer', on='Run')

# estas muestras no se van a tener en cuenta por nuestra query porque estan mal clasificadas
# dfsra_dfinfo_merge_dfena_nodupl.loc[preu['ReleaseDate'].isnull()]['study_accession'].unique()
# principalmente, no tienen bien puesto el organism o strategy en la base de dato SRA


In [4]:
import re
import time
gsm_dic = {}


for index, row in dfsra_dfinfo_merge_dfena_nodupl.iterrows():
    exp = row['Experiment']
    expac = row['experiment_accession']
    putativeGSMs = [str(row['run_alias']),str(row['sample_alias']),str(row['SampleName']),str(row['Library Name'])]
    if 'GSM' in '_'.join(putativeGSMs):
        gsmadd = False
        for i in putativeGSMs:
            if re.match(r'GSM\d+$',i):
                gsmadd = i
                # print(expac,exp,putativeGSMs,gsmadd)
                break
        if not gsmadd:
            for i in putativeGSMs:
                if 'GSM' in i:
                    gsmadd = i.split('_')[0]
                    # print(expac,exp,putativeGSMs,gsmadd)
                    break

        if str(exp) != 'nan' and gsmadd:
            gsm_dic[gsmadd] = exp
        else:
            if str(expac) != 'nan' and gsmadd:
                gsm_dic[gsmadd] = expac
            else:
                print(expac,exp,putativeGSMs, 'NOTPARAMETRI')



LISTNOENCOTRADOS = []
for index, row in dfjose.iterrows():
    if 'GSM' in row.Experiment:
        try:
            dfjose['Experiment'] = dfjose['Experiment'].replace(row.Experiment, gsm_dic[row.Experiment])
#             gsm_dic[row.Experiment]
        except KeyError:
            LISTNOENCOTRADOS.append(row.Experiment)



In [6]:
counter = 0
for index, row in dfsra_dfinfo_merge_dfena_nodupl.iterrows():
    if row['experiment_accession']!=row['Experiment']:
        if str(row['experiment_accession']) != 'nan':
            if re.match(r'SRX\d+$',row['experiment_accession']):
                counter+=1
                # print(index, row.Experiment, row.experiment_accession)
                dfsra_dfinfo_merge_dfena_nodupl.at[index,'Experiment'] = row.experiment_accession
                # dfsra_dfinfo_merge_dfena_nodupl['Experiment'] = dfsra_dfinfo_merge_dfena_nodupl['Experiment'].replace(row.Experiment, row.experiment_accession)
                
            else:
                print(row.experiment_accession)


In [7]:
dfsra_dfinfo_dfena_nodupl_merge_dfjose =  pd.merge(dfsra_dfinfo_merge_dfena_nodupl, dfjose, how='left', on='Experiment')
dfweb.rename(columns={'Unnamed: 0':'BioProject','0':'Project Tittle','1':'Project Description','2':'Project Biblio'}, inplace = True)
dfsra_dfinfo_dfena_nodupl_dfjose_merge_dfweb =  pd.merge(dfsra_dfinfo_dfena_nodupl_merge_dfjose, dfweb, how='outer', on='BioProject')


In [8]:
len(dfsra_dfinfo_dfena_nodupl_merge_dfjose)

94235

In [9]:
import numpy as np
count=0
count2=0
for col in list(dfsra_dfinfo_dfena_nodupl_merge_dfjose.columns):
    if '_x' in col:
        count2+=1
        col = col[:-2]
        removecol = True
        comparison_column = np.where(dfsra_dfinfo_dfena_nodupl_merge_dfjose[f"{col}_x"] == dfsra_dfinfo_dfena_nodupl_merge_dfjose[f"{col}_y"], True, False)
        # for val in comparison_column:
        #     if val != True:
        #         removecol = False 
        if removecol:
            count+=1
            dfsra_dfinfo_dfena_nodupl_merge_dfjose.drop(columns=[f"{col}_y"],inplace=True)
            dfsra_dfinfo_dfena_nodupl_merge_dfjose.rename(columns={f"{col}_x": f"{col}"},inplace=True)
            print(dfsra_dfinfo_dfena_nodupl_merge_dfjose.shape)
        else:
            testdf = dfsra_dfinfo_dfena_nodupl_merge_dfjose
            testdf['compa'] = comparison_column
            testdf = testdf.loc[testdf['compa'] == False]
            print(testdf[[f"{col}_x" ,f"{col}_y", "compa"]])
                
            print(f'dfsra_dfinfo_dfena_nodupl_merge_dfjose.drop(columns=["{col}_y"])')
            print(f'dfsra_dfinfo_dfena_nodupl_merge_dfjose.rename(columns=["{col}_y": "{col}"])')



(94235, 113)
(94235, 112)


In [13]:
dfsra_dfinfo_dfena_nodupl_merge_dfjose = dfsra_dfinfo_dfena_nodupl_merge_dfjose.drop(dfsra_dfinfo_dfena_nodupl_merge_dfjose[(dfsra_dfinfo_dfena_nodupl_merge_dfjose.LibrarySelection == 'MNase')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.LibrarySelection == 'Restriction Digest')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.LibrarySelection == 'padlock probes capture metho')].index)
dfsra_dfinfo_dfena_nodupl_merge_dfjose = dfsra_dfinfo_dfena_nodupl_merge_dfjose.drop(dfsra_dfinfo_dfena_nodupl_merge_dfjose[(dfsra_dfinfo_dfena_nodupl_merge_dfjose.LibrarySource == 'GENOMIC')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.LibrarySource == 'METAGENOMIC')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.LibrarySource == 'TRANSCRIPTOMIC SINGLE CELL')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.LibrarySource == 'METATRANSCRIPTOMIC')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.LibrarySource == 'SYNTHETIC')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.LibrarySource == 'VIRAL RNA')].index)
dfsra_dfinfo_dfena_nodupl_merge_dfjose = dfsra_dfinfo_dfena_nodupl_merge_dfjose.drop(dfsra_dfinfo_dfena_nodupl_merge_dfjose[(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Platform == 'LS454')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Platform == 'OXFORD_NANOPORE')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Platform == 'HELICOS')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Platform == 'PACBIO_SMRT')].index)
dfsra_dfinfo_dfena_nodupl_merge_dfjose = dfsra_dfinfo_dfena_nodupl_merge_dfjose.drop(dfsra_dfinfo_dfena_nodupl_merge_dfjose[(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Model == 'PacBio RS')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Model == 'Helicos HeliScope')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Model == 'Illumina HiSeq X')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Model == 'HiSeq X Five')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Model == 'Illumina iSeq 100')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Model == 'NextSeq 2000')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Model == 'DNBSEQ-T7')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Model == 'GridION')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Model == 'PromethION')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Model == 'Sequel II')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Model == 'NextSeq 1000')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Model == 'DNBSEQ-G400')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Model == '454 GS FLX+')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Model == 'MGISEQ-2000RS')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Model == 'PacBio RS II')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Model == 'Sequel')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Model == '454 GS')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Model == '454 GS FLX Titanium')|(dfsra_dfinfo_dfena_nodupl_merge_dfjose.Model == 'MinION')].index)


In [ ]:
dfFINAL=dfsra_dfjose_dfinfo_dfena_merge_dfweb.reindex(columns=['Run','Experiment','Sample','BioSample','SRAStudy','BioProject','Submission',
                                                                 'run_alias','experiment_alias','sample_alias','ScientificName',
                                                                 'experiment_title','sample_title','study_title','description','sample_description',
                                                                 'Project Tittle','Project Description','Project Biblio',
                                                                 'LibraryStrategy','LibrarySelection','LibrarySource','LibraryLayout','Platform','Model'])